Capstone Project Week 1

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [2]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('ISO_3166-1_alpha-2.html', 'w') as fo:
    fo.write(article)

In [15]:
from bs4 import BeautifulSoup
#! pip install beautifulsoup4
# Load article, turn into soup and get the <table>s.
article = open('ISO_3166-1_alpha-2.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:5] == ['Postcode', 'Borough', 'Neighbhourhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('iso_3166-1_alpha-2_codes.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode, Borough, Neighbhourhood = [td.text.strip() for td in tds[:3]]
        # Wikipedia does something funny with country names containing
        # accented characters: extract the correct string form.
        if '!' in Postcode:
            Postcode = Postcode[Postcode.index('!')+1:]
        print('; '.join([Postcode, Borough, Neighbhourhood]), file=fo)

In [7]:
df = pd.read_csv('iso_3166-1_alpha-2_codes.txt' , sep=';' , header=None)



In [8]:
df.head(10)
 


,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [10]:
df.columns=['Postcode', 'Borough', 'Neighbhourhood']
df.head(10)

,Postcode,Borough,Neighbhourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [28]:
df1=df[df.Borough != ' Not assigned']
#df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df1.head(10)

,Postcode,Borough,Neighbhourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [32]:
#df1.Neighbhourhood=df.Borough[df.Neighbhourhood == ' Not assigned']
df1['Neighbhourhood'] = df1['Borough'].where(df1['Neighbhourhood'] == ' Not assigned',df1['Neighbhourhood'])
df1.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Neighbhourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [33]:
df1.shape

(211, 3)

In [39]:
df2 = df1.groupby(['Postcode','Borough'])['Neighbhourhood'].apply(', '.join).reset_index()
df2

,Postcode,Borough,Neighbhourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village ..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [40]:
df2.shape

(211, 3)

Question 2

In [44]:
!pip install geocoder
import geocoder # import geocoder



     |████████████████████████████████| 102kB 17.3MB/s ta 0:00:01
     |████████████████████████████████| 829kB 32.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 16.2MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


NameError: name 'postal_code' is not defined

In [46]:
df3 = pd.read_csv('https://cocl.us/Geospatial_data' )


In [48]:
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
df2.rename(columns={'Postcode':'Postal Code',}, 
                 inplace=True)
df2.head()

,Postal Code,Borough,Neighbhourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [50]:
df4=pd.merge(df2, df3, on='Postal Code')

In [51]:
df4.head()

,Postal Code,Borough,Neighbhourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Question 3

AttributeError: module 'pandas' has no attribute 'df4'

In [52]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [56]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbhourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [59]:
DowntownToronto_data = df4[df4['Borough'] == ' Downtown Toronto'].reset_index(drop=True)
DowntownToronto_data.head(10)

,Postal Code,Borough,Neighbhourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union S...",43.640816,-79.381752


In [60]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto  are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto  are 43.6541737, -79.3808116451341.


In [62]:
# create map of Manhattan using latitude and longitude values
map_DownToro = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DowntownToronto_data['Latitude'], DowntownToronto_data['Longitude'], DowntownToronto_data['Neighbhourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DownToro)  
    
map_DownToro

In [63]:
CLIENT_ID = 'T1RU1KZV5ZS0ZBZGI5HBXMYWVTOJX5XIUMNGA3KYIQ2L3AOI' # your Foursquare ID
CLIENT_SECRET = 'ONNESP1AWAKF2UNIKEDSVIPE3O02CGMNCKVVX3550VGH23BB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T1RU1KZV5ZS0ZBZGI5HBXMYWVTOJX5XIUMNGA3KYIQ2L3AOI
CLIENT_SECRET:ONNESP1AWAKF2UNIKEDSVIPE3O02CGMNCKVVX3550VGH23BB


In [65]:
DowntownToronto_data.loc[0, 'Neighbhourhood']

' Rosedale'

In [67]:
neighborhood_latitude = DowntownToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DowntownToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DowntownToronto_data.loc[0, 'Neighbhourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Rosedale are 43.6795626, -79.37752940000001.


In [75]:
search_query = 'Indian'
radius = 500
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Indian .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=T1RU1KZV5ZS0ZBZGI5HBXMYWVTOJX5XIUMNGA3KYIQ2L3AOI&client_secret=ONNESP1AWAKF2UNIKEDSVIPE3O02CGMNCKVVX3550VGH23BB&ll=43.6541737,-79.3808116451341&v=20180604&query=Indian&radius=500&limit=30'

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4f865ce97dfb002c463c84'},
 'response': {'venues': [{'id': '4afd920ff964a520ad2822e3',
    'name': 'Indian Biriyani House',
    'location': {'address': '181 Dundas St W',
     'crossStreet': 'W of Chestnut St',
     'lat': 43.65511996683289,
     'lng': -79.3866447102921,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65511996683289,
       'lng': -79.3866447102921}],
     'distance': 481,
     'postalCode': 'M5G 1C7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['181 Dundas St W (W of Chestnut St)',
      'Toronto ON M5G 1C7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1565492828',
  

In [77]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4afd920ff964a520ad2822e3,Indian Biriyani House,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1565492828,False,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",481,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O..."
1,4b2a634af964a52020a824e3,Indian Flavour,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1565492828,False,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",312,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront..."
2,5165c333e4b07a7ad88d8a69,Joe's Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1565492828,False,NaN,NaN,43.658150,-79.381563,"[{'label': 'display', 'lat': 43.65814977325445...",446,NaN,CA,NaN,NaN,Canada,[Canada]
3,4fea497c121d2480d3579412,Indian Biriyani House,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1565492828,False,120 Adelaide Street West,at Bay Street,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",459,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor..."
4,596f9dea1fa7632be2eacab9,Mami's Indian Cuisine,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1565492828,False,NaN,NaN,43.656986,-79.385840,"[{'label': 'display', 'lat': 43.65698561897954...",511,M5G 2N2,CA,Toronto,ON,Canada,"[Toronto ON M5G 2N2, Canada]"


In [78]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Indian Biriyani House,Indian Restaurant,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",481,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O...",4afd920ff964a520ad2822e3
1,Indian Flavour,Indian Restaurant,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",312,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront...",4b2a634af964a52020a824e3
2,Joe's Indian Restaurant,Indian Restaurant,NaN,NaN,43.658150,-79.381563,"[{'label': 'display', 'lat': 43.65814977325445...",446,NaN,CA,NaN,NaN,Canada,[Canada],5165c333e4b07a7ad88d8a69
3,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,at Bay Street,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",459,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",4fea497c121d2480d3579412
4,Mami's Indian Cuisine,Food Truck,NaN,NaN,43.656986,-79.385840,"[{'label': 'display', 'lat': 43.65698561897954...",511,M5G 2N2,CA,Toronto,ON,Canada,"[Toronto ON M5G 2N2, Canada]",596f9dea1fa7632be2eacab9
5,Touch - Indian Cuisine,Indian Restaurant,1 Adelaide St. E,at Yonge St,43.649869,-79.378218,"[{'label': 'display', 'lat': 43.64986899674456...",522,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",4d137f8c7a8ba1433cb3d169


In [79]:
dataframe_filtered.name

0      Indian Biriyani House
1             Indian Flavour
2    Joe's Indian Restaurant
3      Indian Biriyani House
4      Mami's Indian Cuisine
5     Touch - Indian Cuisine
Name: name, dtype: object

In [80]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
DowntownToronto_venues = getNearbyVenues(names=DowntownToronto_data['Neighbhourhood'],
                                   latitudes=DowntownToronto_data['Latitude'],
                                   longitudes=DowntownToronto_data['Longitude']
                                  )

 Rosedale
 Cabbagetown,  St. James Town
 Church and Wellesley
 Harbourfront,  Regent Park
 Ryerson,  Garden District
 St. James Town
 Berczy Park
 Central Bay Street
 Adelaide,  King,  Richmond
 Harbourfront East,  Toronto Islands,  Union Station
 Design Exchange,  Toronto Dominion Centre
 Commerce Court,  Victoria Hotel
 Harbord,  University of Toronto
 Chinatown,  Grange Park,  Kensington Market
 CN Tower,  Bathurst Quay,  Island airport,  Harbourfront West,  King and Spadina,  Railway Lands,  South Niagara
 Stn A PO Boxes 25 The Esplanade
 First Canadian Place,  Underground city
 Christie
